In [1]:
import os, random, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.stats import mode

In [2]:
def remove_row(d_set):
    tmp_d_set = d_set.drop_duplicates()
    if tmp_d_set[tmp_d_set.duplicated('WW')].shape[0] > 0:
        tmp_d_set = tmp_d_set.drop_duplicates('WW')#, keep='last')
    tmp_d_set = tmp_d_set.sort_values(by='WW')
    tmp_d_set = tmp_d_set.reset_index(drop=True)
    return tmp_d_set

In [3]:
f_list = os.listdir('./data1/')
f_list.sort()

product_file = [i for i, v in enumerate(f_list) if ('E61D' not in v.split('_')) & ('Inven' not in v.split('_'))]
sale_file = [i for i, v in enumerate(f_list) if 'E61D' in v.split('_')]
inven_file = [i for i, v in enumerate(f_list) if 'Inven' in v.split('_')]

print(f_list)
print(inven_file)
print(product_file)
print(sale_file)


['02.Detail_생산계획 준수율.csv', 'DynamicResult.csv', 'E61D_MKT_FCST_2018_10_2019_03.csv', 'E61D_MKT_FCST_2019_01_2019_12.csv', 'E61D_MKT_FCST_2019_10_2020_03.csv', 'E61D_MKT_FCST_2020_01_2020_12.csv', 'E61D_MKT_FCST_2020_10_2021_03.csv', 'Inven_3xxC.csv', 'Inven_3xxT.csv', 'Inven_5xx0.csv']
[7, 8, 9]
[0, 1]
[2, 3, 4, 5, 6]


## Inven ##

In [4]:
for i in inven_file:
    if '3xxC' in f_list[i]:
        tmp_3xxC = pd.read_csv('./data1/'+f_list[i])
        tmp_3xxC.columns = list(['Week', '3xxC'])
    elif '3xxT' in f_list[i]:
        tmp_3xxT = pd.read_csv('./data1/'+f_list[i])
        tmp_3xxT.columns = list(['Week', '3xxT'])                                                  
    else :
        tmp_5xxx = pd.read_csv('./data1/'+f_list[i])
        tmp_5xxx.columns = list(['Week', '5xxx'])
        
worknum = pd.DataFrame({'Week' : list(set(tmp_3xxC['Week']) | set(tmp_3xxT['Week']) | set(tmp_5xxx['Week']))})
tmp_inven_5xxx = pd.merge(worknum, tmp_5xxx, how='left', on='Week')
tmp_inven_3xxT = pd.merge(worknum, tmp_3xxT, how='left', on='Week')
tmp_inven_3xxC = pd.merge(worknum, tmp_3xxC, how='left', on='Week')
tmp_inven = tmp_inven_5xxx.iloc[2:].copy().reset_index(drop=True)
tmp_inven['3xxC'] = tmp_inven_3xxC['3xxC'].iloc[:-2].reset_index(drop=True)
tmp_inven['3xxT'] = tmp_inven_3xxT['3xxT'].iloc[1:-1].reset_index(drop=True)
tmp_inven['Inven'] = tmp_inven.iloc[:,1:].sum(axis=1)
tmp_ww = np.array(tmp_inven['Week'])
tmp_inven_prev = []
for i in range(0, len(tmp_ww)):
    tmp_prev_inven = []
    for j in range(1,9):
        if i-j < 0 :
            tmp_prev_inven.append(-999)
        else:
            tmp_prev_inven.append(tmp_inven.iloc[(i-j),4])
    tmp_inven_prev.append(tmp_prev_inven)
tmp_inven_prev = pd.DataFrame(tmp_inven_prev)
inven_prev_name = list(map(lambda x : 'Prev_Inven_'+str(x), np.arange(1, 9, 1)))
tmp_inven_prev.columns = inven_prev_name
tmp_inven = pd.concat([tmp_inven, tmp_inven_prev], axis=1)
tmp_inven.to_csv('E61D_Inven.csv', index=False)

## Product ##

In [76]:
for m in product_file:
    tmp_product = pd.read_csv('./data1/'+f_list[m])       
    if 'Dynamic' in f_list[m]:      
        plan_ww_unique = tmp_product['Plan Date'].unique()
        plan_ww_unique.sort()
        target_ww_unique = tmp_product['Date'].unique()
        target_ww_unique.sort()

        for i in range(0, len(plan_ww_unique)):
            tmp_post = []
            for j in range(1, 13):
                if i+j < len(plan_ww_unique) :
                    if tmp_product[(tmp_product['Plan Date']==plan_ww_unique[i]) & (tmp_product['Date']==(plan_ww_unique[i+j]))].shape[0] > 0:
                        tmp_post_val = tmp_product[(tmp_product['Plan Date']==plan_ww_unique[i]) &(tmp_product['Date']==(plan_ww_unique[i+j]))].iloc[0,2]
                        tmp_post.append(tmp_post_val)
                    else:
                        tmp_post.append(-999)
                else:
                    tmp_post.append(-999)                    
            if i == 0:
                tmp_product_post = tmp_post
            else:
                tmp_product_post = np.vstack((tmp_product_post, tmp_post))
        product_post = pd.DataFrame(tmp_product_post)
        product_post_name = list(map(lambda x : 'Post_Product_'+str(x), np.arange(1, 13, 1)))
        product_post.columns = product_post_name        
        product_post['WW'] = plan_ww_unique

        plan_ww_unique = tmp_product['Plan Date'].unique()
        plan_ww_unique.sort()
        target_ww_unique = tmp_product['Date'].unique()
        target_ww_unique.sort()

        for i in range(0, len(target_ww_unique)):
            tmp_var = []
            for j in range(1, 13):
                index_col = i-j if i-j > 0 else 0

                if tmp_product[(tmp_product['Date']==target_ww_unique[i]) & (tmp_product['Plan Date']==(target_ww_unique[index_col]))].shape[0] > 0:
                    tmp_var_val = tmp_product[(tmp_product['Date']==target_ww_unique[i]) &(tmp_product['Plan Date']==(target_ww_unique[index_col]))].iloc[0,2]
                    tmp_var.append(tmp_var_val)
                else:
                    tmp_var.append(-999)
            if i == 0:
                tmp_product_var = tmp_var
            else:
                tmp_product_var = np.vstack((tmp_product_var, tmp_var))

        product_var = pd.DataFrame(tmp_product_var)
        product_var_name  = list(map(lambda x : 'Var_Product_'+str(x),  np.arange(1, 13, 1)))
        product_var.columns = product_var_name
        product_var['WW'] = target_ww_unique    
    else:          
        tmp_product = tmp_product.sort_values(by=['Plan Week'], axis=0)
        product_set = tmp_product[['Plan Week', 'Result Qty_x']].copy()        
        product_set.columns = list(['Week', 'Product'])

tmp_product_ww = tmp_inven['Week'] # product_set['Week']
for i in range(0, len(tmp_product_ww)):
    tmp_prev = []
    for j in range(1,13):
        if product_set[product_set['Week'] == (tmp_product_ww[i]-j)].shape[0]>0:
            tmp_prev_val = product_set[product_set['Week'] == (tmp_product_ww[i]-j)].iloc[0,1]
            tmp_prev.append(tmp_prev_val)
        else:
            tmp_prev.append(-999)
    if i == 0:
        product_prev = tmp_prev
    else:
        product_prev = np.vstack((product_prev, tmp_prev))
product_prev = pd.DataFrame(product_prev)
product_prev_name  = list(map(lambda x : 'Prev_Product_'+str(x),  np.arange(1, 13, 1)))
product_prev.columns = product_prev_name        
product_prev['WW'] = tmp_product_ww

product = pd.merge(pd.merge(pd.merge(pd.merge(worknum, product_set, how='left', on='Week'), 
                                    product_var, how='left', left_on='Week', right_on='WW'), 
                    product_post, how='left', left_on='Week', right_on='WW'), 
        product_prev, how='left', left_on='Week', right_on='WW')
ww_col = [z for i,z in enumerate(product.columns) if 'WW' in z]
product = product.drop(columns = ww_col)
product.to_csv('E61D_Product.csv', index=False)

## Sales ##

In [49]:
#sale_file
for k in range(0, len(sale_file)):
    tmp_sale = pd.read_csv('./data1/'+f_list[sale_file[k]])    
    tmp_sale_ww = list(set(list(tmp_sale.iloc[:,0])) & set(list(tmp_sale.columns[1:].astype(int))))
    tmp_sale_ww.sort()
    k_sale_var = []
    k_sale_post = []
    k_sale_rslt = []
    for i in range(0, len(tmp_sale_ww)):        
        tmp_sale_var = list([tmp_sale_ww[i]])
        tmp_sale_post= list([tmp_sale_ww[i]])
        tmp_sale_rslt= list([tmp_sale_ww[i]])
        tmp_row_id = tmp_sale.index[tmp_sale.iloc[:,0] == tmp_sale_ww[i]].tolist()[0]
        tmp_col_id = np.where(tmp_sale.columns == str(tmp_sale_ww[i]))[0].item()
        if tmp_sale_ww[i] < tmp_sale_ww[len(tmp_sale_ww)-1-11]:#np.max(tmp_sale_ww) - 11: #(tmp_sale_ww[i] + 1) <= np.max(tmp_sale_ww):            
            for j in range(1,13):            
                if str((tmp_sale_ww[i+j])) in tmp_sale.columns :
                    tmp_sale_post.append(tmp_sale.iloc[tmp_row_id, (tmp_col_id+j)])
                else :
                    tmp_sale_post.append(-999)
        else :            
            if k == (len(sale_file)-1):
                for j in range(1, 13):                            
                    if (tmp_col_id+j) < tmp_sale.shape[1]:
                        tmp_sale_post.append(tmp_sale.iloc[tmp_row_id, (tmp_col_id+j)])
                    else:
                        tmp_sale_post.append(-999)
        for j in range(1,13):
            if (tmp_sale_ww[i]) >= tmp_sale_ww[0+1]:#np.min(tmp_sale_ww)+1:
                tmp_sale_var.append(tmp_sale.iloc[(tmp_row_id-j), tmp_col_id])       
        if tmp_sale_ww[i] <= tmp_sale_ww[len(tmp_sale_ww)-1-1] :#np.max(tmp_sale_ww)-1: #+ 5 
                tmp_sale_rslt.append(mode(tmp_sale.iloc[(tmp_row_id+1):, tmp_col_id]).mode[0])
        if len(tmp_sale_var) > 1:
            k_sale_var.append(tmp_sale_var)
        if len(tmp_sale_post) > 1:
            k_sale_post.append(tmp_sale_post)
        if len(tmp_sale_rslt) > 1:
            k_sale_rslt.append(tmp_sale_rslt)
    if k == 0:
        sale_var = k_sale_var
        sale_post = k_sale_post
        sale_rslt = k_sale_rslt
    else:
        sale_var = np.vstack((sale_var, k_sale_var))
        sale_post = np.vstack((sale_post, k_sale_post))
        sale_rslt = np.vstack((sale_rslt, k_sale_rslt))

sale_post_name = list(map(lambda x : 'Post_Sale_'+str(x), np.arange(1, 13, 1)))
sale_var_name  = list(map(lambda x : 'Var_Sale_'+str(x),  np.arange(1, 13, 1)))
sale_var = pd.DataFrame(sale_var)
sale_post = pd.DataFrame(sale_post)
sale_rslt = pd.DataFrame(sale_rslt)
sale_var.columns = list(['WW']) + sale_var_name
sale_post.columns = list(['WW']) + sale_post_name
sale_rslt.columns = list(['WW', 'Sale'])

sale_rslt1 = remove_row(sale_rslt)
sale_post1 = remove_row(sale_post)
sale_var1 = remove_row(sale_var)        

tmp_sale_ww = sale_rslt1['WW']
for i in range(0, len(tmp_sale_ww)):
    tmp_sale = []    
    for j in range(1,13):
        index_col = i-j if i-j > 0 else 0
        if sale_rslt1[sale_rslt1['WW'] == (tmp_sale_ww[index_col])].shape[0]>0:
            tmp_sale_val = sale_rslt1[sale_rslt1['WW'] == (tmp_sale_ww[index_col])].iloc[0,1]
            tmp_sale.append(tmp_sale_val)
        else:
            tmp_sale.append(-999)
    if i == 0:
        sale_prev = tmp_sale
    else:
        sale_prev = np.vstack((sale_prev, tmp_sale))
sale_prev = pd.DataFrame(sale_prev)
sale_prev_name  = list(map(lambda x : 'Prev_Sale_'+str(x),  np.arange(1, 13, 1)))
sale_prev.columns = sale_prev_name
sale_prev['WW'] = tmp_sale_ww
sale = pd.merge(pd.merge(pd.merge(pd.merge(worknum, sale_rslt1, how='left', left_on='Week', right_on='WW'), 
                          sale_var1, how='left', left_on='Week', right_on='WW'),
                        sale_post1, how='left', left_on='Week', right_on='WW'),
                sale_prev, how='left', left_on='Week', right_on='WW')
ww_col = [z for i,z in enumerate(sale.columns) if 'WW' in z]
sale = sale.drop(columns = ww_col)
sale.to_csv('E61D_Sale.csv', index=False)